- Data Split & Merge

In [ ]:
import Extractor
import pandas as pd
import sqlite3

conn1 = sqlite3.connect('Entertainment.db')
cur1 = conn1.cursor()
df1 = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn1)
conn1.close()

conn2 = sqlite3.connect('Entertainment101.db')
cur2 = conn2.cursor()
df2 = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn2)
conn2.close()

df = df1.append(df2) # 엔터테인먼트는 여기서 분할해서 저장해야함.

df['pk'] = range(1, len(df)+1)

conn3 = sqlite3.connect('Total_Ent.db')
cur3 = conn3.cursor()
cur3.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df.to_sql('head', conn3, if_exists='append', index=False)
conn3.close()

In [ ]:
conn = sqlite3.connect('Total_Ent.db')
cur = conn.cursor()
df = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn)
df1 = df[:2000000]
df2 = df[2000000:4000000]
df3 = df[4000000:6000000]
df4 = df[6000000:8000000]
df5 = df[8000000:]

conn = sqlite3.connect('Ent_1.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df1.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_2.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df2.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_3.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df3.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_4.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df4.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_5.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df5.to_sql('head', conn, if_exists='append', index=False)
conn.close()

- Words & Variables Extraction

In [1]:
import Extractor
import pandas as pd
import sqlite3

conn = sqlite3.connect('News.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head ORDER BY wdate DESC',conn)
conn.close()
df.drop_duplicates(keep='first', inplace=True)
ext = Extractor.Ext(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

# 변수 생성
statistic = ext.extract_statistic_value(sent)
r_rat = ext.extract_r_rat(sent,statistic)
statistic = ext.combine_var(statistic, r_rat)
conn = sqlite3.connect('News_var.db')
statistic.to_sql('var', conn)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 840339 from 931920 sents. mem=0.463 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=5714865, mem=2.219 Gb
[Noun Extractor] batch prediction was completed for 205244 words
[Noun Extractor] checked compounds. discovered 327303 compounds
[Noun Extractor] postprocessing detaching_features : 30191 -> 29999
[Noun Extractor] postprocessing ignore_features : 29999 -> 29781
[Noun Extractor] postprocessing ignore_NJ : 29781 -> 29378
[Noun Extractor] 29378 nouns (327303 compounds) with min frequency=9
[Noun Extractor] flushing was done. mem=2.590 Gb                    
[Noun Extractor] 76.22 % eojeols are covered
training was done. used memory 0.282 Gb0.287 Gb
all cohesion probabilities was computed. # words = 1490
all branching entropies was computed # words = 5642

all accessor variety was computed # words = 35218
training was done. used memory 1.072 Gb1.072 Gb
all cohesion probabilities was computed. # words = 7792
all branching entropies was computed # words = 35382
all accessor variety was computed # words = 35382
training was done. used memory 1.072 Gb1.072 Gb
all cohesion probabilities was computed. # words = 7864
all branching entropies was computed # words = 35756
all accessor variety was computed # words = 35756
training was done. used memory 1.072 Gb1.072 Gb
all cohesion probabilities was computed. # words = 7926
all branching entropies was computed # words = 36089
all accessor variety was computed # words = 36089
training was done. used memory 1.073 Gb1.073 Gb
all cohesion probabilities was computed. # words = 7961
all branching entropies was computed # words = 36317
all accessor variety was computed # words = 36317
training was done. used memory 1.073 Gb1.073 Gb
all cohesion probabilities was computed. # words = 7996
all branching entr

all branching entropies was computed # words = 42425
all accessor variety was computed # words = 42425
training was done. used memory 1.076 Gb1.076 Gb
all cohesion probabilities was computed. # words = 9162
all branching entropies was computed # words = 42567
all accessor variety was computed # words = 42567
training was done. used memory 1.076 Gb1.076 Gb
all cohesion probabilities was computed. # words = 9183
all branching entropies was computed # words = 42706
all accessor variety was computed # words = 42706
training was done. used memory 1.076 Gb1.076 Gb
all cohesion probabilities was computed. # words = 9212
all branching entropies was computed # words = 42851
all accessor variety was computed # words = 42851
training was done. used memory 1.076 Gb1.076 Gb
all cohesion probabilities was computed. # words = 9231
all branching entropies was computed # words = 43021
all accessor variety was computed # words = 43021
training was done. used memory 1.076 Gb1.076 Gb
all cohesion probabil

all cohesion probabilities was computed. # words = 10719
all branching entropies was computed # words = 51425
all accessor variety was computed # words = 51425
training was done. used memory 1.085 Gb1.085 Gb
all cohesion probabilities was computed. # words = 10732
all branching entropies was computed # words = 51499
all accessor variety was computed # words = 51499
training was done. used memory 1.085 Gb1.085 Gb
all cohesion probabilities was computed. # words = 10745
all branching entropies was computed # words = 51571
all accessor variety was computed # words = 51571
training was done. used memory 1.085 Gb1.085 Gb
all cohesion probabilities was computed. # words = 10745
all branching entropies was computed # words = 51620
all accessor variety was computed # words = 51620
training was done. used memory 1.085 Gb1.085 Gb
all cohesion probabilities was computed. # words = 10773
all branching entropies was computed # words = 51752
all accessor variety was computed # words = 51752
training

all accessor variety was computed # words = 54754
training was done. used memory 1.089 Gb1.099 Gb
all cohesion probabilities was computed. # words = 11341
all branching entropies was computed # words = 54823
all accessor variety was computed # words = 54823
training was done. used memory 1.089 Gb1.099 Gb
all cohesion probabilities was computed. # words = 11346
all branching entropies was computed # words = 54874
all accessor variety was computed # words = 54874
training was done. used memory 1.089 Gb1.099 Gb
all cohesion probabilities was computed. # words = 11348
all branching entropies was computed # words = 54984
all accessor variety was computed # words = 54984
training was done. used memory 1.094 Gb1.099 Gb
all cohesion probabilities was computed. # words = 11369
all branching entropies was computed # words = 55065
all accessor variety was computed # words = 55065
training was done. used memory 1.094 Gb1.099 Gb
all cohesion probabilities was computed. # words = 11369
all branching

...

all branching entropies was computed # words = 110332
all accessor variety was computed # words = 110332
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 32939
all branching entropies was computed # words = 110335
all accessor variety was computed # words = 110335
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 32942
all branching entropies was computed # words = 110338
all accessor variety was computed # words = 110338
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 32942
all branching entropies was computed # words = 110339
all accessor variety was computed # words = 110339
'시형씨'
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 32945
all branching entropies was computed # words = 110341
all accessor variety was computed # words = 110341
'마셨다'
training was done. used memory 0.958 Gb0.95

training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33191
all branching entropies was computed # words = 110668
all accessor variety was computed # words = 110668
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33193
all branching entropies was computed # words = 110674
all accessor variety was computed # words = 110674
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33196
all branching entropies was computed # words = 110677
all accessor variety was computed # words = 110677
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33196
all branching entropies was computed # words = 110677
all accessor variety was computed # words = 110677
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33196
all branching entropies was computed # words = 110677
a

training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33452
all branching entropies was computed # words = 110916
all accessor variety was computed # words = 110916
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33456
all branching entropies was computed # words = 110923
all accessor variety was computed # words = 110923
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33460
all branching entropies was computed # words = 110935
all accessor variety was computed # words = 110935
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33462
all branching entropies was computed # words = 110944
all accessor variety was computed # words = 110944
training was done. used memory 0.961 Gb0.961 Gb
all cohesion probabilities was computed. # words = 33466
all branching entropies was computed # words = 110949
a

'어느때'
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 33696
all branching entropies was computed # words = 111205
all accessor variety was computed # words = 111205
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 33697
all branching entropies was computed # words = 111205
all accessor variety was computed # words = 111205
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 33699
all branching entropies was computed # words = 111205
all accessor variety was computed # words = 111205
'확산속'
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 33700
all branching entropies was computed # words = 111208
all accessor variety was computed # words = 111208
'총선판'
training was done. used memory 0.958 Gb0.958 Gb
all cohesion probabilities was computed. # words = 33701
all branching entropies was computed 

MemoryError: 

- Variables Merge

In [4]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('Ent_1_var.db')
df1 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_2_var.db')
df2 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_3_var.db')
df3 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_4_var.db')
df4 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_5_var.db')
df5 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('HP_var.db')
df6 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

df = df1.append(df2).append(df3).append(df4).append(df5).append(df6)
df.to_excel('var.xlsx')